<a href="https://colab.research.google.com/github/Rodio346/open-to-you-to/blob/main/Twitter%20Sentiment%20Ayalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [58]:
import tweepy # for tweet mining
import pandas as pd # for data manipulation and analysis
import numpy as np # for working with arrays and carrying out mathematical operations. Pandas is built on Numpy
import csv # to read and write csv files
import re # In-built regular expressions library
import string # Inbuilt string library
import glob # to retrieve files/pathnames matching a specified pattern. 
import random # generating random numbers
import requests # to send HTTP requests
from PIL import Image # for opening, manipulating, and saving many different image file f
import matplotlib.pyplot as plt # for plotting

# Set the limits for Pandas Dataframe display to avoid potential system freeze
pd.set_option("display.max_rows", 15)
pd.set_option("display.max_columns", 15)
pd.set_option('display.max_colwidth', 40)

# Natural Language Processing Toolkit
from nltk.corpus import stopwords, words # get stopwords from NLTK library & get all words in english language
from nltk.tokenize import word_tokenize # to create word tokens
# from nltk.stem import PorterStemmer (I played around with Stemmer and decided to use Lemmatizer instead)
from nltk.stem import WordNetLemmatizer # to reduce words to orginal form
from nltk import pos_tag # For Parts of Speech tagging


from textblob import TextBlob # TextBlob - Python library for processing textual data

In [59]:
df = pd.read_csv("/content/drive/MyDrive/VS data.csv", index_col=None, header=0) # Add your csv path here 

In [60]:
df

,created_at,id,text,Unnamed: 3,truncated,is_quote_status,retweet_count,favorite_count,favorited,retweeted,lang
0,Mon Dec 06 10:14:04 +0000 2021,1.467799e+18,"@A_mnahh Hello, We are sorry you are...",NaN,True,False,0,0,False,False,en
1,Mon Dec 06 09:18:38 +0000 2021,1.467786e+18,@SaraJoudy Please contact by phone 8...,NaN,True,False,0,0,False,False,en
2,Mon Dec 06 06:32:44 +0000 2021,1.467744e+18,@corpsekiki Hi there! We will gladly...,NaN,False,False,0,1,False,False,en
3,Mon Dec 06 01:17:51 +0000 2021,1.467665e+18,@edwardsh23 We understand you're ups...,NaN,True,False,0,0,False,False,en
4,Mon Dec 06 00:03:19 +0000 2021,1.467646e+18,@DarrykaAshley We've let our team kn...,NaN,False,False,0,0,False,False,en
...,...,...,...,...,...,...,...,...,...,...,...
3245,Wed Sep 09 21:17:24 +0000 2020,1.303805e+18,@CathyMabley We'd be happy to help c...,NaN,True,False,0,1,False,False,en
3246,Wed Sep 09 19:57:50 +0000 2020,1.303785e+18,"@amber_pulliam Hi Amber, we truly ap...",NaN,False,False,0,1,False,False,en
3247,Wed Sep 09 16:58:58 +0000 2020,1.303740e+18,@BagosAriyanto Alana Felisberto!,NaN,False,False,0,1,False,False,es
3248,Wed Sep 09 16:31:34 +0000 2020,1.303733e+18,"@W22693280Ashley We're concerned, an...",NaN,True,False,0,0,False,False,en


In [61]:
# Function to remove punctuations, links, emojis, and stop words
def preprocessTweets(tweet):
    tweet = tweet.lower()  #has to be in place
    # Remove urls
    tweet = re.sub(r"http\S+|www\S+|https\S+", '', tweet, flags=re.MULTILINE)
    # Remove user @ references and '#' from tweet
    tweet = re.sub(r'\@\w+|\#|\d+', '', tweet)
    # Remove stopwords
    tweet_tokens = word_tokenize(tweet)  # convert string to tokens
    filtered_words = [w for w in tweet_tokens if w not in stop_words]
    #filtered_words = [w for w in filtered_words if w not in emojis]
    filtered_words = [w for w in filtered_words if w in word_list]

    # Remove punctuations
    unpunctuated_words = [char for char in filtered_words if char not in string.punctuation]
    unpunctuated_words = ' '.join(unpunctuated_words)

    return "".join(unpunctuated_words)  # join words with a space in between them


# function to obtain adjectives from tweets
def getAdjectives(tweet):
    tweet = word_tokenize(tweet)  # convert string to tokens
    tweet = [word for (word, tag) in pos_tag(tweet)
             if tag == "JJ"]  # pos_tag module in NLTK library
    return " ".join(tweet)  # join words with a space in between them

In [62]:

# Defining my NLTK stop words and my user-defined stop words
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('words')
nltk.download('punkt')
stop_words = list(stopwords.words('english'))
user_stop_words = ['2020', 'year', 'many', 'much', 'amp', 'next', 'cant', 'wont', 'hadnt',
                    'havent', 'hasnt', 'isnt', 'shouldnt', 'couldnt', 'wasnt', 'werent',
                    'mustnt', '’', '...', '..', '.', '.....', '....', 'been…', 'one', 'two',
                    'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten', 'aht',
                    've', 'next']
alphabets = list(string.ascii_lowercase)
stop_words = stop_words + user_stop_words + alphabets
word_list = words.words()  # all words in English language


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [63]:
df['Processed_Tweets'] = df['text'].apply(preprocessTweets)

In [64]:
df


,created_at,id,text,Unnamed: 3,truncated,is_quote_status,retweet_count,favorite_count,favorited,retweeted,lang,Processed_Tweets
0,Mon Dec 06 10:14:04 +0000 2021,1.467799e+18,"@A_mnahh Hello, We are sorry you are...",NaN,True,False,0,0,False,False,en,hello sorry able track order happy h...
1,Mon Dec 06 09:18:38 +0000 2021,1.467786e+18,@SaraJoudy Please contact by phone 8...,NaN,True,False,0,0,False,False,en,please contact phone midnight local ...
2,Mon Dec 06 06:32:44 +0000 2021,1.467744e+18,@corpsekiki Hi there! We will gladly...,NaN,False,False,0,1,False,False,en,hi gladly forward request see lip gl...
3,Mon Dec 06 01:17:51 +0000 2021,1.467665e+18,@edwardsh23 We understand you're ups...,NaN,True,False,0,0,False,False,en,understand upset would love learn pr...
4,Mon Dec 06 00:03:19 +0000 2021,1.467646e+18,@DarrykaAshley We've let our team kn...,NaN,False,False,0,0,False,False,en,let team know love miss vanilla lace...
...,...,...,...,...,...,...,...,...,...,...,...,...
3245,Wed Sep 09 21:17:24 +0000 2020,1.303805e+18,@CathyMabley We'd be happy to help c...,NaN,True,False,0,1,False,False,en,happy help check status refund pleas...
3246,Wed Sep 09 19:57:50 +0000 2020,1.303785e+18,"@amber_pulliam Hi Amber, we truly ap...",NaN,False,False,0,1,False,False,en,hi amber truly appreciate attention ...
3247,Wed Sep 09 16:58:58 +0000 2020,1.303740e+18,@BagosAriyanto Alana Felisberto!,NaN,False,False,0,1,False,False,es,
3248,Wed Sep 09 16:31:34 +0000 2020,1.303733e+18,"@W22693280Ashley We're concerned, an...",NaN,True,False,0,0,False,False,en,concerned would love see locate plea...


In [65]:
# function to return words to their base form using Lemmatizer
def preprocessTweetsSentiments(tweet):
    tweet_tokens = word_tokenize(tweet)
    lemmatizer = WordNetLemmatizer() # instatiate an object WordNetLemmatizer Class
    lemma_words = [lemmatizer.lemmatize(w) for w in tweet_tokens]
    return " ".join(lemma_words)

In [66]:
nltk.download('wordnet')
# Apply preprocessTweetsSentiments function to the 'Processed Tweets' column to generate a new column
# called 'Processed_Tweets'
df['Tweets_Sentiments'] =df['Processed_Tweets'].apply(preprocessTweetsSentiments)
df.head() # Check dataframe first 5 rows

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,created_at,id,text,Unnamed: 3,truncated,is_quote_status,retweet_count,favorite_count,favorited,retweeted,lang,Processed_Tweets,Tweets_Sentiments
0,Mon Dec 06 10:14:04 +0000 2021,1.467799e+18,"@A_mnahh Hello, We are sorry you are...",NaN,True,False,0,0,False,False,en,hello sorry able track order happy h...,hello sorry able track order happy h...
1,Mon Dec 06 09:18:38 +0000 2021,1.467786e+18,@SaraJoudy Please contact by phone 8...,NaN,True,False,0,0,False,False,en,please contact phone midnight local ...,please contact phone midnight local ...
2,Mon Dec 06 06:32:44 +0000 2021,1.467744e+18,@corpsekiki Hi there! We will gladly...,NaN,False,False,0,1,False,False,en,hi gladly forward request see lip gl...,hi gladly forward request see lip gl...
3,Mon Dec 06 01:17:51 +0000 2021,1.467665e+18,@edwardsh23 We understand you're ups...,NaN,True,False,0,0,False,False,en,understand upset would love learn pr...,understand upset would love learn pr...
4,Mon Dec 06 00:03:19 +0000 2021,1.467646e+18,@DarrykaAshley We've let our team kn...,NaN,False,False,0,0,False,False,en,let team know love miss vanilla lace...,let team know love miss vanilla lace...


In [67]:


# Create function to obtain Subjectivity Score
def getSubjectivity(tweet):
    return TextBlob(tweet).sentiment.subjectivity

# Create function to obtain Polarity Score
def getPolarity(tweet):
    return TextBlob(tweet).sentiment.polarity

# Create function to obtain Sentiment category
def getSentimentTextBlob(polarity):
    if polarity < 0:
        return "Negative"
    elif polarity == 0:
        return "Neutral"
    else:
        return "Positive"



In [68]:


# Apply all functions above to respective columns
df['Subjectivity']=df['Tweets_Sentiments'].apply(getSubjectivity)
df['Polarity']=df['Tweets_Sentiments'].apply(getPolarity)
df['Sentiment']=df['Polarity'].apply(getSentimentTextBlob)

# See quick results of the Sentiment Analysis
df['Sentiment'].value_counts()

Positive    1898
Neutral      821
Negative     531
Name: Sentiment, dtype: int64

In [69]:
# Create dataframe for Count of Sentiment Categories
bar_chart = df['Sentiment'].value_counts().rename_axis('Sentiment').to_frame('Total Tweets').reset_index()


In [70]:
bar_chart

,Sentiment,Total Tweets
0,Positive,1898
1,Neutral,821
2,Negative,531


In [72]:
df.head()

,created_at,id,text,Unnamed: 3,truncated,is_quote_status,retweet_count,...,retweeted,lang,Processed_Tweets,Tweets_Sentiments,Subjectivity,Polarity,Sentiment
0,Mon Dec 06 10:14:04 +0000 2021,1.467799e+18,"@A_mnahh Hello, We are sorry you are...",NaN,True,False,0,...,False,en,hello sorry able track order happy h...,hello sorry able track order happy h...,0.875,0.266667,Positive
1,Mon Dec 06 09:18:38 +0000 2021,1.467786e+18,@SaraJoudy Please contact by phone 8...,NaN,True,False,0,...,False,en,please contact phone midnight local ...,please contact phone midnight local ...,0.000,0.000000,Neutral
2,Mon Dec 06 06:32:44 +0000 2021,1.467744e+18,@corpsekiki Hi there! We will gladly...,NaN,False,False,0,...,False,en,hi gladly forward request see lip gl...,hi gladly forward request see lip gl...,1.000,0.500000,Positive
3,Mon Dec 06 01:17:51 +0000 2021,1.467665e+18,@edwardsh23 We understand you're ups...,NaN,True,False,0,...,False,en,understand upset would love learn pr...,understand upset would love learn pr...,0.600,0.500000,Positive
4,Mon Dec 06 00:03:19 +0000 2021,1.467646e+18,@DarrykaAshley We've let our team kn...,NaN,False,False,0,...,False,en,let team know love miss vanilla lace...,let team know love miss vanilla lace...,0.400,0.333333,Positive


In [73]:
df.to_csv('VS data updated.csv')